<a href="https://colab.research.google.com/github/1017Kosuke/About-me-/blob/main/cifar10_keras_alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# connect with GitHub

全体の構造の流れ

1.   画像の前処理の定義
2.   モデルの構成

GCNやSZAを用いる
ランダムな画像を設定して精度を高める
モデルに関しては高APIのライブラリから参照してモデルを最初は構築し、その後にモデルのチューニングを行う

In [ ]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm
from PIL import Image
from sklearn.model_selection import train_test_split

#学習データ
x_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture05/data/x_train.npy')
t_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture05/data/t_train.npy')

#テストデータ
x_test = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture05/data/x_test.npy')

class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train):
        data = x_train.astype('float32')
        self.x_train = []
        for i in range(data.shape[0]):
            self.x_train.append(Image.fromarray(np.uint8(data[i])))
        self.t_train = t_train
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.x_train)

    def __getitem__(self, idx):
        return self.transform(self.x_train[idx]), torch.tensor(t_train[idx], dtype=torch.long)

class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test):
        data = x_test.astype('float32')
        self.x_test = []
        for i in range(data.shape[0]):
            self.x_test.append(Image.fromarray(np.uint8(data[i])))
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.x_test)

    def __getitem__(self, idx):
        return self.transform(self.x_test[idx])

trainval_data = train_dataset(x_train, t_train)
test_data = test_dataset(x_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#ライブラリの定義
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation, Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical
!pip install --upgrade scikeras scikit-learn
# KerasRegressorではなくKerasClassifierを使用
from scikeras.wrappers import KerasClassifier # Changed from KerasRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

In [ ]:
print(np.shape(x_train))
print(np.shape(t_train))
print(np.shape(x_test))
# (50000, 32, 32, 3)
# (50000,)
# (10000, 32, 32, 3)

(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)


In [ ]:
#モデルの定義（簡単のため、Pythorchを使って実装する）
def build_model(activation , optimizer):
    model = Sequential()

    model.add(Conv2D(input_shape=(32, 32, 3), filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1,1)))

    model.add(Flatten())
    model.add(Dense(256, activation=activation, input_dim=784))
    model.add(Dense(128, activation=activation))
    model.add(Dense(64, activation=activation))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [ ]:
# ハイパーパラメータを準備
activation = ['relu', 'sigmoid']
optimizer = ['adam', 'sgd']
nb_epoch = [10, 20]
batch_size = [64, 128, 256]

# グリッドサーチ対象のハイパーパラメーターを辞書型にまとめる
param_grid = dict(activation=activation, optimizer=optimizer,nb_epoch=nb_epoch, batch_size=batch_size)

# モデルを作成
model = KerasClassifier(model = build_model, verbose=False)

# グリッドサーチの実行
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(x_train, t_train)

AttributeError: 'super' object has no attribute '__sklearn_tags__'